In [2]:
import pandas as pd

#given_training_data_set
df = pd.read_csv("train.csv")


#target
#rename the 'bg+1:00' column to 'target'
df.rename(columns={'bg+1:00': 'target'}, inplace=True)


#i tend to save the df after each step in my preprocessing so that I can debug from the point, not the entire ds
###
####DataFrame to a new CSV file
file_path = 'train_bg_renamed.csv'
df.to_csv(file_path, index=False)

#just to see the new df 
df = pd.read_csv(file_path)

#activity columns
activity_cols = [col for col in df.columns if col.startswith('activity-')]

#unique values from all activity columns
unique_values = pd.unique(df[activity_cols].values.ravel('K'))

#total number of unique values
print(f"Total number of unique values in activity columns: {len(unique_values)}")

# Print the list of unique values
print("Unique values in activity columns:")
print(unique_values)

#here i mapped the different activity based on their intensity,
###like the walking is less intense then the running, so wal is mapped to numerial value of 3 and run as 8
#the activity-to-intensity mapping
activity_intensity_mapping = {
    'Walk': 3,
    'Indoor climbing': 3,
    'Yoga': 2,
    'Zumba': 7,
    'HIIT': 9,
    'Dancing': 3,
    'Swim': 7,
    'Outdoor Bike': 5,
    'Aerobic Workout': 6,
    'Sport': 5,
    'Walking': 3,
    'Running': 8,
    'Swimming': 7,
    'Run': 8,
    'Weights': 4,
    'Workout': 5,
    'Tennis': 6,
    'Strength training': 5,
    'Stairclimber': 6,
    'Spinning': 6,
    'Hike': 7,
    'Bike': 7
}

####cols
activity_cols = [col for col in df.columns if col.startswith('activity-')]

##replacing the intensity score by their map
df[activity_cols] = df[activity_cols].replace(activity_intensity_mapping)

#replacing if any zerros here
df[activity_cols] = df[activity_cols].fillna(0)

for col in activity_cols:
    if not pd.api.types.is_numeric_dtype(df[col]):
        df.drop(columns=[col], inplace=True)
        print(f"Dropped column '{col}' due to non-numeric values.")


#one activity_col_representing_all mapped score        
df['activity_score'] = df[activity_cols].sum(axis=1)

# dropping all, keeping only 'activity_score'
df.drop(columns=activity_cols, inplace=True)



####   new CSV file
df.to_csv('train_with_activity_score.csv', index=False)

####okkk lets look if the str is replaced by the int
print(df[activity_cols].head())


######lets see againnn
df


####latest csv from yesterday (same dir: mac/from_sapelo2/kaggle/glucose_pred/preprocessing_oct_28.ipynb)
#
#
####
###
file_path = 'train_with_activity_score.csv'  # Replace with your actual cleaned dataset file path
df = pd.read_csv(file_path)

###the carb list
carbs_cols = [col for col in df.columns if col.startswith('carbs-')]

##replacing the nan or blank by 0 as it is giving error in the EDA 
df[carbs_cols] = df[carbs_cols].fillna(0)

### instead of 5 min i am doing the total cab value for the 15 minute interval to reduce the features
for i in range(0, len(carbs_cols), 3):
    # Create the new column name for the 15-minute interval
    new_col_name = f'carbs_{carbs_cols[i]}_to_{carbs_cols[i+2]}'
    
    # Sum the values of three consecutive columns to represent a 15-minute interval
    df[new_col_name] = df[carbs_cols[i]] + df[carbs_cols[i+1]] + df[carbs_cols[i+2]]

#drop original carbss
df.drop(columns=carbs_cols, inplace=True)

#updated DataFrame
df.to_csv('train_with_15_min_carbs_intervals.csv', index=False)

#verify if there is anything or any error
print(df.head())


####the df from the last preprocessing step
import pandas as pd

#cleaned dataset upto now
file_path = 'train_with_15_min_carbs_intervals.csv'  
df = pd.read_csv(file_path)

### similarly the calorie to 15 minute interval instead of 5 minute interval
## i am using the same function

def aggregate_to_15_min_intervals(df, column_prefix):
    
    cols = [col for col in df.columns if col.startswith(column_prefix)]
    
    ###nan and empty by 0
    df[cols] = df[cols].fillna(0)
    
    # grouping
    for i in range(0, len(cols), 3):
        ## 15 interval from 5
        new_col_name = f'{column_prefix}_from_{cols[i]}_to_{cols[i+2]}'
        
        # now i will get the total carb in those columns 
        df[new_col_name] = df[cols[i]] + df[cols[i+1]] + df[cols[i+2]]
    
    #drop original 5 min interval columns, so droping the regular 72 
    df.drop(columns=cols, inplace=True)

    return df

# cal columns aggreg
df = aggregate_to_15_min_intervals(df, 'cals')

# similarly for the steps to get total steps before the blood glucose at 0:00
df = aggregate_to_15_min_intervals(df, 'steps')

###new CSV file
df.to_csv('train_with_15_min_intervals_cals_steps.csv', index=False)

#### lets verify 
print(df.head())


# ## latest cleaned dataset 

import pandas as pd
import numpy as np


file_path = 'train_with_15_min_intervals_cals_steps.csv'  
df = pd.read_csv(file_path)

# ### bg columns
bg_cols = [col for col in df.columns if col.startswith('bg-')]

### Function to impute missing values ###
def impute_bg_values(row, bg_cols):
    # fforward fill: Handle the first few columns by propagating forward the next available value
    if pd.isna(row[bg_cols[0]]):  # check point: the first column is NaN
        for i in range(1, len(bg_cols)):
            if not pd.isna(row[bg_cols[i]]):
                row[bg_cols[0]] = row[bg_cols[i]]
                break
    
    ##looping through each bg column (starting from the second one)
    for i in range(1, len(bg_cols)):
        if pd.isna(row[bg_cols[i]]):
            if i == len(bg_cols) - 1:
                #### last column: Impute with the previous column's value
                row[bg_cols[i]] = row[bg_cols[i-1]]
            else:
                ###all the middle : impute with the mean of the previous and next column
                if not pd.isna(row[bg_cols[i-1]]) and not pd.isna(row[bg_cols[i+1]]):
                    row[bg_cols[i]] = (row[bg_cols[i-1]] + row[bg_cols[i+1]]) / 2
                elif not pd.isna(row[bg_cols[i-1]]):
                    row[bg_cols[i]] = row[bg_cols[i-1]]
                elif not pd.isna(row[bg_cols[i+1]]):
                    row[bg_cols[i]] = row[bg_cols[i+1]]
    
    return row

#lets execute the impute fun 
df = df.apply(lambda row: impute_bg_values(row, bg_cols), axis=1)

### just checking the df if there is any missing 
missing_values_count = df[bg_cols].isnull().sum()
print(missing_values_count)

####  UPDATED DF 
df.to_csv('train_with_imputed_bg_final_corrected.csv', index=False)

df[bg_cols].isnull().sum().sum()
df
import pandas as pd

#latest dataset
file_path = 'train_with_imputed_bg_final_corrected.csv'  
df = pd.read_csv(file_path)

###now lets work on the  hr columns
hr_cols = [col for col in df.columns if col.startswith('hr-')]

#fillling NaN values with the mean of the column for each hr column
df[hr_cols] = df[hr_cols].fillna(df[hr_cols].mean())

### updated Data_Frame
df.to_csv('train_with_imputed_hr_final_mean.csv', index=False)

### any remaining missing values
missing_values_count = df[hr_cols].isnull().sum()
print(missing_values_count)

### just to verify the changes
print(df[hr_cols].head())


df[hr_cols].isnull().sum().sum()

df
import pandas as pd

### latest cleaned dataset
file_path = 'train_with_imputed_hr_final_mean.csv' 
df = pd.read_csv(file_path)

## insuling cols 72 cols 
insulin_cols = [col for col in df.columns if col.startswith('insulin-')]

### missing values in insulin columns
missing_values_insulin = df[insulin_cols].isnull().sum()

## count of the missing in insulin 
print(missing_values_insulin)

### ok total miss 
total_missing_insulin = missing_values_insulin.sum()
print(f"Total missing values in insulin columns: {total_missing_insulin}")



# latest df same before the explore on the missing vals 
file_path = 'train_with_imputed_bg_final_corrected.csv'  
df = pd.read_csv(file_path)

### insulin columns  
insulin_cols = [col for col in df.columns if col.startswith('insulin-')]

# impute function missing values for insulin columns row-wise
def impute_insulin_values(row, insulin_cols):
    ### copy to put it back 
    row_insulin = row[insulin_cols].copy()

    ### missing values at the beginning (forward fill)
    first_valid_index = row_insulin.first_valid_index()
    if first_valid_index is not None:
        first_valid_index_loc = insulin_cols.index(first_valid_index)
        if first_valid_index_loc > 0:
            row_insulin.iloc[:first_valid_index_loc] = row_insulin.iloc[first_valid_index_loc]

    #missing values at the end (backward fill)
    last_valid_index = row_insulin.last_valid_index()
    if last_valid_index is not None:
        last_valid_index_loc = insulin_cols.index(last_valid_index)
        if last_valid_index_loc < len(row_insulin) - 1:
            row_insulin.iloc[last_valid_index_loc + 1:] = row_insulin.iloc[last_valid_index_loc]

    #  ## interpolate missing values in between
    row_insulin.interpolate(method='linear', inplace=True)

    return row_insulin

## row-wise imputation function to each row
df[insulin_cols] = df.apply(lambda row: impute_insulin_values(row, insulin_cols), axis=1)

# # handle the case where an entire row of insulin values is missing
def column_mean_imputation(df, insulin_cols):
    for col in insulin_cols:
        # indices where the entire row is missing
        missing_indices = df[df[col].isnull()].index
        
        for idx in missing_indices:
            # mean of 20 rows above and 20 rows below the missing row, if they exist
            start_idx = max(0, idx - 20)
            end_idx = min(len(df), idx + 20)
            
            # mean of the specified range, excluding the missing value itself
            surrounding_mean = df[col].iloc[start_idx:end_idx].mean()
            
            # impute the missing value with the surrounding mean
            df.at[idx, col] = surrounding_mean

# column-wise imputation for rows that are entirely missing
column_mean_imputation(df, insulin_cols)

# # just if there are any remaining missing values
missing_values_count = df[insulin_cols].isnull().sum()
print("Remaining missing values after imputation:", missing_values_count)

#updated DataFrame
df.to_csv('train_with_imputed_insulin_final_corrected_1.csv', index=False)

# irst few rows to verify the changes
print(df[insulin_cols].head())

print(df[insulin_cols].mean())
import pandas as pd

#atest dataset
file_path = 'train_with_imputed_insulin_final_corrected_1.csv'  
df = pd.read_csv(file_path)

# hr columns
hr_cols = [col for col in df.columns if col.startswith('hr-')]

#NaN values with the mean of the column for each hr column
df[hr_cols] = df[hr_cols].fillna(df[hr_cols].mean())

# updated DataFrame
df.to_csv('train_preprocessed.csv', index=False)

# Check if there are any remaining missing values
missing_values_count = df[hr_cols].isnull().sum()
print(missing_values_count)

# Display the first few rows to verify the changes
print(df[hr_cols].head())


/tmp/ipykernel_518129/3279662898.py:4: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train.csv")
/tmp/ipykernel_518129/3279662898.py:14: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Total number of unique values in activity columns: 23
Unique values in activity columns:
[nan 'Walk' 'Indoor climbing' 'Yoga' 'Zumba' 'HIIT' 'Dancing' 'Swim'
 'Outdoor Bike' 'Aerobic Workout' 'Sport' 'Walking' 'Running' 'Swimming'
 'Run' 'Weights' 'Workout' 'Tennis' 'Strength training' 'Stairclimber'
 'Spinning' 'Hike' 'Bike']
   activity-5:55  activity-5:50  activity-5:45  activity-5:40  activity-5:35  \
0            0.0            0.0            0.0            0.0            0.0   
1            0.0            0.0            0.0            0.0            0.0   
2            0.0            0.0            0.0            0.0            0.0   
3            0.0            0.0            0.0            0.0            0.0   
4            0.0            0.0            0.0            0.0            0.0   

   activity-5:30  activity-5:25  activity-5:20  activity-5:15  activity-5:10  \
0            0.0            0.0            0.0            0.0            0.0   
1            0.0            0.

/tmp/ipykernel_518129/3279662898.py:76: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


      id p_num      time  bg-5:55  bg-5:50  bg-5:45  bg-5:40  bg-5:35  \
0  p01_0   p01  06:10:00      NaN      NaN      9.6      NaN      NaN   
1  p01_1   p01  06:25:00      NaN      NaN      9.7      NaN      NaN   
2  p01_2   p01  06:40:00      NaN      NaN      9.2      NaN      NaN   
3  p01_3   p01  06:55:00      NaN      NaN      8.7      NaN      NaN   
4  p01_4   p01  07:10:00      NaN      NaN      8.4      NaN      NaN   

   bg-5:30  bg-5:25  ...  cals-0:35  cals-0:30  cals-0:25  cals-0:20  \
0      9.7      NaN  ...        NaN        NaN        NaN        NaN   
1      9.2      NaN  ...        NaN        NaN        NaN        NaN   
2      8.7      NaN  ...        NaN        NaN        NaN        NaN   
3      8.4      NaN  ...        NaN        NaN        NaN        NaN   
4      8.1      NaN  ...        NaN        NaN        NaN        NaN   

   cals-0:15  cals-0:10  cals-0:05  cals-0:00  target  activity_score  
0        NaN        NaN        NaN        NaN    13.4   

/tmp/ipykernel_518129/3279662898.py:176: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[cols[i]] + df[cols[i+1]] + df[cols[i+2]]
/tmp/ipykernel_518129/3279662898.py:176: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[cols[i]] + df[cols[i+1]] + df[cols[i+2]]
/tmp/ipykernel_518129/3279662898.py:176: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.

      id p_num      time  bg-5:55  bg-5:50  bg-5:45  bg-5:40  bg-5:35  \
0  p01_0   p01  06:10:00      NaN      NaN      9.6      NaN      NaN   
1  p01_1   p01  06:25:00      NaN      NaN      9.7      NaN      NaN   
2  p01_2   p01  06:40:00      NaN      NaN      9.2      NaN      NaN   
3  p01_3   p01  06:55:00      NaN      NaN      8.7      NaN      NaN   
4  p01_4   p01  07:10:00      NaN      NaN      8.4      NaN      NaN   

   bg-5:30  bg-5:25  ...  steps_from_steps-2:25_to_steps-2:15  \
0      9.7      NaN  ...                                  0.0   
1      9.2      NaN  ...                                  0.0   
2      8.7      NaN  ...                                  0.0   
3      8.4      NaN  ...                                  0.0   
4      8.1      NaN  ...                                  0.0   

   steps_from_steps-2:10_to_steps-2:00  steps_from_steps-1:55_to_steps-1:45  \
0                                  0.0                                  0.0   
1           